# Bioinfo Projekt Gruppe 4-4: k-means 
*Members: Benedict, Julia, Thorge and Marilena*  

## Tasks

Implement the following algorithms in python using the numpy library:

1. implement k-means:

1. compare your implementation with the sklearn implementation with respect to quality and speed
  implement mini-batch k-means:

1. compare your implementation with the sklearn implementation with respect to quality and speed
  implement k-means++ initialization:

1. Compare the runtime and quality of your k-means implementation and your mini batch k-means implementation for  different datasets. You can use code from sklearn to generate datasets of arbitrary size and difficulty  (https://scikit-learn.org/stable/auto_examples/cluster/plot_cluster_comparison.html#sphx-glr-auto-examples-cluster-plot-cluster-comparison-py). You should generate multiple plots to visualize the comparison (eg. you can plot the runtinme / cluster quality for different dataset sizes / number of clusters)

1. Cluster the 3K PBMCs from a Healthy Donor Dataset from 10x Genomics
1. use scanpy to load the data ( see https://scanpy-tutorials.readthedocs.io/en/latest/pbmc3k.html)
1. compare the performance of your implementations with the sklearn implementations

## Import

First of all a few packages are imported that were used throughout the project

In [ ]:
import random
from statistics import mean
from sklearn.datasets.samples_generator import make_blobs
import matplotlib.pyplot as plt
import urllib.request as url
import numpy as np
import scanpy as sc
import pandas as pd
import tarfile
import csv
from sklearn.base import BaseEstimator, ClusterMixin, TransformerMixin

## Class Kmeans

The class Kmeans() is based on the sklearn variant.  
It takes following arguments:  
     **inits** --> initialisations  
     Standard = 10    
        **.k** --> number of clusters  
        Standard = 8  
    **maxit** --> maximum iterations.  
    Standard = 300  
    **method** --> method of choosing starting clusters.  
    Standard = "++". Option = "rng"

In [ ]:
class Kmeans(BaseEstimator, ClusterMixin, TransformerMixin):               # Input: processed dataset, Output: clustered data (kmeans, kmeans++)
    def __init__(self, inits=10, k=8, maxit=300, method="++"):
        
        self.labels_ = None
        self.cluster_centers_ = None
        self._inits = inits
        self._k = k
        self._maxit = maxit
        self._method = method

#### fit method:

fits the data

In [ ]:
def fit(self,data):
        self._data = data
        best_clust = float('inf')
        
        for i in (range(self._inits)):
            dot = np.random.choice(range(len(self._data)), self._k, replace=False)
            self.cluster_centers_ = self._data[dot]
            for i in range(self._maxit):
                clusters = np.expand_dims(self.cluster_centers_, axis=1)
                data = np.expand_dims(self._data, axis=0)
                eucl = np.linalg.norm(clusters-data, axis=2) # euclidean dist by using integrated numpy function
                self.labels_ = np.argmin(eucl, axis = 0)
                for i in range(self._k): # range of clusters
                    position = np.where(self.labels_ == i) # position im array bestimmen und dann die entspechenden punkte aus data auslesen
                    self.cluster_centers_[i] = self._data[position].mean(axis = 0)
                    #out = pd.DataFrame(data[np.argwhere(dist == i)].squeeze())
                overall_quality = np.sum(np.min(eucl.T, axis=1))
                if overall_quality < best_clust:
                    best_clust = overall_quality
                    best_dist = self.labels_
                    best_centers = self.cluster_centers_
            self.cluster_centers_ = best_centers
            self.labels_ = best_dist
                
        return self

#### Now expanded by kmeans++  
set argument: method="++" (by default as in sklearn)

    Instead of all centroids only the first is appended by random choice.

__The remaining k-1 centroids are chosen as followed:__ 

**1.** Calculate squared distance D of every point to its clostest centroid  
**2.** Every point is assigned a probability to be chosen as the next centroid according to:

$$D(x)\over \sum^{}_{x\in X} D(x)$$  

**3.** New centroid is picked from all datapoints considering their assigned probabilities  
**4.** Repeat steps 1-3 until k centroids are chosen

    Note: This method only provides intial centroids and does not change the clustering process for the following iterations


In [ ]:
            if self._method == "rng": # random centers are choosen
                #print("rng")
                dot = np.random.choice(range(len(self._data)), self._k, replace=False)
                self.cluster_centers_ = self._data[dot]
            elif self._method == "++": # kmeans++ is initiated
                #print("++")
                dot = np.random.choice(len(self._data), replace=False) # random startpunkt
                clusters = np.array([self._data[dot]])
                pointer = np.array([])
                for i in range (self._k-1):
                    D = np.array([])
            
                    for j in range (len(self._data)):
                        D = np.append(D,np.min(np.sum((self._data[j]-clusters)**2, axis = 1)))
                
                    pointer = np.append(pointer, D, axis = 0) 
            
                    p = D/np.sum(D)
                    cummulative_p = np.cumsum(p)
            
                    r = random.random()
                    ind = np.where(cummulative_p >= r)[0][0]
            
                    clusters = np.append(clusters,[self._data[ind]], axis = 0)
                self.cluster_centers_ = clusters
            else:
                raise AttributeError("No valid method")

#### predict method:
used to predict which point depends to a certain cluster

In [ ]:
def predict(self, X):
        clusters = np.expand_dims(self.cluster_centers_, axis=1)
        data = np.expand_dims(X, axis=0)
        eucl = np.linalg.norm(clusters-data, axis=2) # euclidean dist by using integrated numpy function
        self.labels_ = np.argmin(eucl, axis = 0)
        return self.labels_ #returns the cluster with minimum distance

#### transform method

transforms data to cluster-distance space

In [ ]:
def transform(self, X):
        clusters = np.expand_dims(self.cluster_centers_, axis=1)
        data = np.expand_dims(X, axis=0)
        eucl = np.linalg.norm(clusters-data, axis=2)
        return eucl.T

## Class MiniBatch Kmeans

In [ ]:
import numba

The class MiniBatch Kmeans() takes following arguments:    
     **inits** --> initialisations  
     Standard = 10    
     **.k** --> number of clusters  
     Standard = 8  
     **max_iterations** --> maximum iterations.  
     Standard = 300  
     **tol** --> (tolerated percantage of samples changing cluster after one iteration.  
     Standard = 1e-3  
     **batch_size** --> number of random samples of the data for the Mini Batch.  
     Standard = 128

In [ ]:
class MiniBatchKMeans:  # Input: processed dataset, Output: clustered data (MiniBatchKMeans)           
    def __init__(self, k=8, inits=300, max_iterations=300, tol=1e-3, batch_size=128):
          self._k = k
        self._inits = inits
        self._max_iterations = max_iterations
        self._tol = tol
        self._batch_size = batch_size

#### create_batch method:

chooses x (x= batch_size) random data points from data, to create the data_batch

In [ ]:
    def create_batch(self, data): 
        data_batch = np.random.choice(range(len(data)), self._batch_size, replace=False)
        return data[data_batch]

#### initalize method:

chooses k random data points from data, to set centers for clustering

In [ ]:
   def initialize(self, data): 
        indices = np.random.choice(range(len(data)), self._k, replace=False)
        return data[indices]

#### expectation method:

measures the euclidean distance between each data_batch points and center points

In [ ]:
    def expectation(self, data, centroids): 
        centroids = np.expand_dims(centroids, axis=1)
        data_batch = np.expand_dims(data, axis=0)
        metric = np.linalg.norm(centroids - data_batch, axis=2)# euclidean dist by using integrated numpy function
        return np.argmin(metric, axis=0)#ordnet die data_batch Punkte den centroids zu

#### maximization method:

For every assigned data point of the batch it moves the centroid in direction of the new barycentre according to the learning rate.

In [ ]:
    @staticmethod
    @numba.jit(nopython=True)
    def _maximization_aux(data, assignments, centroids, centroid_count): 
        update = centroids.copy()
        for idx, assignment in enumerate(assignments):
            data_point = data[idx]
            centroid_count[assignment] += 1
            lr = 1 / centroid_count[assignment] #learning rate, increases with centroid counts
            update[assignment] = update[assignment] * (1 - lr) + data_point * lr
        return update

This part applies maximization_aux on the data using maximization_aux

In [ ]:
 def maximization(self, data, assignments, centroids, centroid_count): 
        return MiniBatchKMeans._maximization_aux(data, assignments, centroids, centroid_count)

#### final_assignments method:

Assignes the rest of the data points to the centroids, which were determined before (not only batch_points)

In [ ]:
    def final_assignments(self, data, centroids): 
        assignments = []
        for idx in range(len(data) // self._batch_size + 1):
            start = idx * self._batch_size
            stop = min(idx * self._batch_size + self._batch_size, len(data))
            sub_result = self.expectation(data[start:stop], centroids)
            assignments.append(sub_result)
        return np.concatenate(assignments, axis=0)

#### fit method:

fits the data 

In [ ]:
    def fit(self, data): 
        centroids, counts = self.initialize(data)
        
        old_centroids = None
        for idx in range(self._max_iterations):
            old_centroids = centroids.copy()
            
            batch = self.create_batch(data)
            assignments = self.expectation(batch, centroids)
            centroids = self.maximization(batch, assignments, centroids, counts)
            
            if np.linalg.norm(centroids - old_centroids) < self._tol: #stops when smaller than tolerance
                break

        result = self.final_assignments(data, centroids)
                
        return centroids, result